# Introduction to parallel programming with python

As previously discussed parallel computing involves solving the same problem with 1 or more cpu cores or even more than one computers. Parallel solutions mostly involve sub tasks that are able to run independently of each other with very little or no communication.

In python the two common approaches to parallel programming are running code with multiple threads or multiple processes. The main difference between the terms is that threads are a part of a process, that is a process may contain one or more threads, but a thread cannot contain a process. Read more on the differences [here](https://www.backblaze.com/blog/whats-the-diff-programs-processes-and-threads/)

Threads from the reading have shared memory while processes have completely memory locations (distributed memory). Using threads can easily lead to to race conditions when data access isn't synchronized, this where a multiple threads try to read/write to the same memory location. Depending on the relative speeds of the threads the results for the same operation could differ.

![alt text](http://twimgs.com/ddj/images/article/2011/0611/thread.gif)

With the above example the final value of x can be 1,2,3 depending how the threads run.

A safer approach is to run multiple process where each will run its own memory space. The trade off here is the communication overhead between separate process.

Another reason why processes may be a better option over threads is python's GIL (Global Interpreter Lock). In order to prevent conflicts between threads python implements a locking mechanism that prevents multiple threads from executing python bytecode at once.The reason for this is beyond this course but for the curious souls read more [here](https://realpython.com/python-gil/)


# Python's `multiprocessing` module

Python has a standard library for writing multi-process programs. Find the official documentation [here](https://docs.python.org/3.6/library/multiprocessing.html)

# The `process` class
The basic approach is to use the `Process` class. Processes are spawned by creating a `Process` object then calling its start method. 

The syntax of a process method is `Process(group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)`.  

The most important arguments include:
*   target - The function/method to be called by the process
*   name - A name for the process
*   args - arguments to pass to the target function




In [1]:
# Example from python's documentation

#import the Process from the multiprocessing library
from multiprocessing import Process

# Function to be run by the process. 
# Prints hello and the any argument passed in the name parameter
def f(name):
    print('hello', name)

# Create a process object which will call the function "f" and passes the string "bob"
p = Process(target=f, args=('bob',))

# Calling start on the process starts its execution
p.start()

# The join method hints the main process to wait for the child process to finish executing before continuing
p.join()

hello bob



Lets look at another example suppose we are to generate 5 random numbers between 1 and 50. We could generate one number after the other or we could do this faster by starting 5 process with each generating one.

In [2]:
from multiprocessing import Process,current_process
import random
import time

# method that generates and prints a random number along with the current process' name
def generate_random_number():
  # sleeping the process for 2 seconds to simiulate some long running computation
  time.sleep(2)
  
  x = random.randint(1,51)
  current_process_name = current_process().name
  print(current_process_name +" says %d"%(x) )
  

# Setup a list of processes that we want to run
processes = [Process(target=generate_random_number, name="process %d"%(x),) for x in range(5)]

# processes now contains a list of process object,
print(processes)

# Run processes
for p in processes:
    p.start()
    
# # Exit the completed processes
for p in processes:
    p.join()

[<Process(process 0, initial)>, <Process(process 1, initial)>, <Process(process 2, initial)>, <Process(process 3, initial)>, <Process(process 4, initial)>]
process 0 says 46
process 1 says 14
process 2 says 24
process 3 says 7
process 4 says 35


In this example we create 5 process. Each runs generating and printing a single random number. The order of the results is based on the order of completion.

# The `Pool` Class
The pool class is similar to the Process class except you control a pool of process. The process in the pool execute the tasks submitted to it.This is a good class if your functions need to return some values.
Please review the official [documentation](https://docs.python.org/3.6/library/multiprocessing.html#module-multiprocessing.pool)

The class has 4 very interesting methods
*   `Pool.apply` 
*   `Pool.apply_async`
*   `Pool.map`
*   `Pool.map_async`

These functions are equivalents of the built in [apply](https://docs.python.org/2/library/functions.html#apply) and [map](https://docs.python.org/3.6/library/functions.html#map) functions

Each method is used based on the problem you need to solve. See the table below
<table>
  <tr>
    <th>Function</th>
    <th>Multiple arguments</th>
    <th>Concurrence</th>
    <th>Blocking</th>
    <th>Ordered-results</th>
  </tr>
  <tr>
    <th>map</th>
    <td>no</td>
    <td>yes</td>
    <td>yes</td>
    <td>yes</td>
  </tr>
  <tr>   
    <th>apply</th>
    <td>yes</td>
    <td>no</td>
    <td>yes</td>
    <td>no</td></tr>
  <tr>  
    <th>map_async</th>
    <td>no</td>
    <td>yes</td>
    <td>no</td>
    <td>yes</td></tr>
  <tr>   
    <th>apply_async</th>
    <td>no</td>
    <td>yes</td>
    <td>no</td>
    <td>no</td></tr>
</table>

The `map_async` will be used often because we are mostly apply the same function hence no need for mutiple arguments, it doesn't block the program until completion and it also returns its results ordered.

Lets take a simple example where we want to find the largest prime factors of numbers from 1 - 1000.

In [3]:
import multiprocessing as mp

# Function to find the largest prime factor
def findLargestPrimeFactor(n):
    primeFactor = 1
    i = 2

    while i <= n / i:
        if n % i == 0:
            primeFactor = i
            n /= i
        else:
            i += 1

    if primeFactor < n:
        primeFactor = n

    return primeFactor

# Let's test out the function with an input of 6. The expected output is 3
print(findLargestPrimeFactor(6))


pool = mp.Pool(processes=4)

result_with_apply = [pool.apply_async(findLargestPrimeFactor, args=(x,)) for x in range(188554848645154545454,188554848645154545464)]
result_with_map = pool.map_async(findLargestPrimeFactor, range(188554848645154545454,188554848645154545464))

output_with_apply = [p.get() for p in result_with_apply]
output_with_map = result_with_map.get()

print(output_with_apply)
print(output_with_map)

factors =[findLargestPrimeFactor(x) for x in range(188554848645154545454,188554848645154545464)]
print (factors)

pool.close()
pool.join()

3.0
[454280053.0, 22463559509.0, 454280053.0, 4601592362484248.0, 454280053.0, 2717112653.0, 454280053.0, 16507698034.0, 454280053.0, 10868450612.0]
[454280053.0, 22463559509.0, 454280053.0, 4601592362484248.0, 454280053.0, 2717112653.0, 454280053.0, 16507698034.0, 454280053.0, 10868450612.0]
[454280053.0, 22463559509.0, 454280053.0, 4601592362484248.0, 454280053.0, 2717112653.0, 454280053.0, 16507698034.0, 454280053.0, 10868450612.0]


# Task
Complete the section below. 

Using 40 concurrent processes,
1.   Generate 100 random numbers between 1 million and 1 billion
2.   Find the smallest prime factor of each number. Function for finding smallest prime factor is provided



In [4]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

Number of processors:  2


In [5]:
import multiprocessing as mp
import random
import time

# Function to find the smallest prime factor
def findSmallestPrimeFactor(n):
    factor = 2 # start at the lowest possible factor
    while n % factor != 0: # go until factor is a factor
        factor += 1 # test the next factor
    return factor
  
# method that generates a random number along with the current process' name
def generate_random_number():
# Todo: complete this section
    
    time.sleep(2)
    x = random.randint(1000000,1000000000)
    return x
# method that get the hundred random and prints a random number
def generate_numbers(x):
    List_random_numbers =[]    
    for i in range(x):
        List_random_numbers.append(generate_random_number())
    return List_random_numbers
# Todo: generate numbers
# Setup a list of processes that we want to run
pool = mp.Pool(processes=40)

result_with_apply = [pool.apply_async(generate_random_number) for _ in range(100)]
List_of_number= [p.get() for p in result_with_apply]


print(List_of_number)
# Todo: find smallest prime factor of each number
result_with_map1 = pool.map_async(findSmallestPrimeFactor, List_of_number)
SmallestPrimeFactors= result_with_map1.get()

pool.close()
pool.join()

print(SmallestPrimeFactors)


[956145377, 379036589, 52013386, 914947912, 317927314, 155341126, 412311084, 1488367, 970078289, 627124006, 457509509, 860857789, 184882926, 102147434, 19075471, 363707071, 927438713, 715889920, 254750227, 204483210, 383170492, 102323477, 481549002, 158508930, 339910645, 848747036, 899596908, 72038688, 780441473, 942724758, 895251478, 943078816, 760784786, 145182460, 397864675, 476197476, 265137297, 747775540, 748864822, 581371434, 222178704, 879130168, 376366527, 813910921, 844918009, 624616927, 804496129, 210282587, 244143580, 535485573, 895096707, 251756681, 106905532, 309515504, 392759882, 256566882, 613692742, 835436176, 715828725, 75281896, 402948693, 617026105, 322610321, 41788853, 114870449, 759488372, 849414483, 769881004, 982331065, 572750352, 332770766, 824910258, 400115995, 831868702, 962475758, 346982598, 922621347, 762327345, 234028910, 267192077, 945582447, 906895451, 394655507, 149370214, 42395973, 15691338, 123448958, 338046239, 552669195, 432452362, 78179690, 67702788